1. Выполнить импорт необходимых библиотек:

In [12]:
!pip3 install tensorflow adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.4 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 6.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip


In [21]:
import numpy as np
import tensorflow as tf
from art.attacks.poisoning.backdoor_attack_dgm.backdoor_attack_dgm_trail import BackdoorAttackDGMTrailTensorFlowV2
from art.estimators.gan.tensorflow import TensorFlowV2GAN
from art.estimators.generation.tensorflow import TensorFlowV2Generator
from art.estimators.classification.tensorflow import TensorFlowV2Classifier
np.random.seed(100)
tf.random.set_seed(100)


In [28]:
### Обход ошибки SSL CERTIFICATE для пунтка 6
import requests
requests.packages.urllib3.disable_warnings()
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

2. Создать класс для модели-генератора изображений

In [23]:
def make_generator_model(capacity: int, z_dim: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Dense(capacity * 7 * 7 * 4, use_bias=False,input_shape=(z_dim,)))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Reshape((7, 7, capacity * 4)))
  assert model.output_shape == (None, 7, 7, capacity * 4)

  model.add(tf.keras.layers.Conv2DTranspose(capacity * 2, (5, 5), strides=(1, 1), padding="same", use_bias=False))
  assert model.output_shape == (None, 7, 7, capacity * 2)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(capacity, (5, 5), strides=(2, 2), padding="same", use_bias=False))
  assert model.output_shape == (None, 14, 14, capacity)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding="same", use_bias=False))

  model.add(tf.keras.layers.Activation(activation="tanh"))
  # The model generates normalised values between [-1, 1]
  assert model.output_shape == (None, 28, 28, 1)

  return model

3. Создать класс для модели-дискриминатора изображений

In [24]:
def make_discriminator_model(capacity: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Conv2D(capacity, (5, 5), strides=(2, 2),padding="same", input_shape=[28, 28, 1]))
  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Conv2D(capacity * 2, (5, 5), strides=(2, 2), padding="same"))
  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1))

  return model

4. Создать атакующий триггер

In [25]:
# Create attacker trigger
z_trigger = np.random.randn(1, 100).astype(np.float64)

5. Создать цель атаки

In [26]:
x_target = np.random.randint(low=0, high=256, size=(28, 28, 1)).astype("float64")
x_target = (x_target - 127.5) / 127.5

6. Загрузить датасет MNIST

In [29]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")
train_images = (train_images - 127.5) / 127.5

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

11490434/11490434 [==============================] - 2s 0us/step


7. Определить функцию потерь дискриминатора

In [30]:
def discriminator_loss(true_output, fake_output):
  true_loss = cross_entropy(tf.ones_like(true_output), true_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  tot_loss = true_loss + fake_loss
  return tot_loss

8. Определить функцию потерь генератора

In [34]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

noise_dim = 100
capacity = 64
generator = TensorFlowV2Generator(encoding_length=noise_dim, model=make_generator_model(capacity, noise_dim))
discriminator_classifier = TensorFlowV2Classifier(model=make_discriminator_model(capacity), nb_classes=2, input_shape=(28, 28, 1))

9. Создать генератор ( на маке грустно:( , тут я ушел уже на гугл коллаб)

In [35]:
gan = TensorFlowV2GAN(generator=generator,
                      discriminator=discriminator_classifier,
                      generator_loss=generator_loss,
                      generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
                      discriminator_loss=discriminator_loss,
                      discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
                      )

10. Создать атаку на генератор

In [ ]:
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)

print("Poisoning estimator")
poisoned_generator = gan_attack.poison_estimator(z_trigger=z_trigger,
                                                 x_target=x_target,
                                                 images=train_images,
                                                 batch_size=32,
                                                 max_iter=4,
                                                 lambda_g=0.1,
                                                 verbose=2)
print("Finished poisoning estimator")

11. Оценить точность атаки

In [45]:
x_pred_trigger = poisoned_generator.model(z_trigger)[0]
print("Target Fidelity (Attack Objective): %.2f%%" %
      np.sum((x_pred_trigger - x_target) ** 2))

Target Fidelity (Attack Objective): 93.7%


12. Сохранить артефакты атаки

In [ ]:
np.save("z_trigger_trail.npy", z_trigger)
np.save("x_target_trail.npy", x_target)
poisoned_generator.model.save("trail-mnist-dcgan")

13.Повторить эксперимент для целевого изображения выбранного из train_image[29] и сгенерированного триггера из диапазона [0;88]

In [43]:
x_target_d = x_target[29:30]
x_target_d = (x_target - 127.5) / 127.5 #Нормализация целевого изображения
z_trigger_d = np.random.randn(1, 88).astype(np.float64)

noise_dim = 88
generator = TensorFlowV2Generator(encoding_length=noise_dim, model=make_generator_model(capacity, noise_dim))

gan = TensorFlowV2GAN(generator=generator,
                      discriminator=discriminator_classifier,
                      generator_loss=generator_loss,
                      generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
                      discriminator_loss=discriminator_loss,
                      discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
                      )

gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)

print("Оценщик отравления")
poisoned_generator = gan_attack.poison_estimator(z_trigger=z_trigger_d,
                                                 x_target=x_target_d,
                                                 images=train_images,
                                                 batch_size=32,
                                                 max_iter=4,
                                                 lambda_g=0.1,
                                                 verbose=2)
print("Законченный оценщик отравления")

x_pred_trigger = poisoned_generator.model(z_trigger_d)[0]
print("Target Fidelity (Attack Objective): %.2f%%" %
      np.sum((x_pred_trigger - x_target_d) ** 2))


Poisoning estimator
 Finished poisoning estimator
Target Fidelity (Attack Objective): 25.66%


14. Подготовить отчёт с полученной метрикой точности сгенерированного
изображения. Сделать выводы об изученном методе проведения атаки на GAN.

### В ходе данной практики применялся метод Retraining with Distillation (ReD). Этот метод переобучения фокусируется исключительно на внутренних уровнях (обучение генератора), не производя изменений в архитектуре. При сравнении двух экспериментов можно сделать вывод, что целевая фидельность второго эксперимента улучшилась относительно первого

## Целевая фидельность для первого эксперимента составляет 93.7%
## Целевая фидельность для второго эксперимента составляет 25.66%.